# Spark SQL

### Introduction

In this lesson, we'll demo how to query spark with Spark SQL.

### Getting Started

In [ ]:
# You may need to reinstall pyspark in colab

!pip install fsspec --quiet
!pip install s3fs --quiet
!pip install pyspark --quiet

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
import pandas as pd
url = "s3://jigsaw-labs-student/houston_claims.csv"
df = pd.read_csv(url)


After loading our data from s3, we can create our dataframe.

In [ ]:
claims_df = spark.createDataFrame(df.astype(str))

And then create a temporary view, which will allow us to query our claims as if it were a table.

In [7]:
claims_df.createOrReplaceTempView("claims")

Let's look at the claims schema.

In [8]:
claims_df.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- reportedCity: string (nullable = true)
 |-- dateOfLoss: string (nullable = true)
 |-- elevatedBuildingIndicator: string (nullable = true)
 |-- floodZone: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- lowestFloodElevation: string (nullable = true)
 |-- amountPaidOnBuildingClaim: string (nullable = true)
 |-- amountPaidOnContentsClaim: string (nullable = true)
 |-- yearofLoss: string (nullable = true)
 |-- reportedZipcode: string (nullable = true)
 |-- id: string (nullable = true)



And now we can simply query spark like sql.  Remember that to execute the job, we need to call an action like `show`.

In [9]:
spark.sql("SELECT * FROM claims ORDER BY yearofLoss LIMIT 3;").show(vertical = True)

-RECORD 0-----------------------------------------
 Unnamed: 0                | 2469                 
 reportedCity              | HOUSTON              
 dateOfLoss                | 1985-11-17T00:00:... 
 elevatedBuildingIndicator | False                
 floodZone                 | C                    
 latitude                  | 29.8                 
 longitude                 | -95.5                
 lowestFloodElevation      | nan                  
 amountPaidOnBuildingClaim | 0.0                  
 amountPaidOnContentsClaim | 0.0                  
 yearofLoss                | 1985-01-01T00:00:... 
 reportedZipcode           | 77024                
 id                        | 5e398d7074cbd479f... 
-RECORD 1-----------------------------------------
 Unnamed: 0                | 2468                 
 reportedCity              | HOUSTON              
 dateOfLoss                | 1985-09-29T00:00:... 
 elevatedBuildingIndicator | False                
 floodZone                 | C 

> Now we can also use a WHERE clause.

In [19]:
spark.sql("SELECT * FROM claims WHERE latitude = 29.8 LIMIT 2;").show(vertical = True)

-RECORD 0-----------------------------------------
 _c0                       | 2                    
 reportedCity              | HOUSTON              
 dateOfLoss                | 2004-06-28 20:00:00  
 elevatedBuildingIndicator | false                
 floodZone                 | X                    
 latitude                  | 29.8                 
 longitude                 | -95.6                
 lowestFloodElevation      | null                 
 amountPaidOnBuildingClaim | 1420.89              
 amountPaidOnContentsClaim | 0.0                  
 yearofLoss                | 2003-12-31 19:00:00  
 reportedZipcode           | 77042                
 id                        | 5e398d6774cbd479f... 
-RECORD 1-----------------------------------------
 _c0                       | 3                    
 reportedCity              | HOUSTON              
 dateOfLoss                | 2009-04-27 20:00:00  
 elevatedBuildingIndicator | false                
 floodZone                 | X 

And if you want to see what is occurring underneath, you can use the `explain` command.

In [20]:
spark.sql("SELECT * FROM claims WHERE latitude = 29.8 LIMIT 2;").explain()

== Physical Plan ==
CollectLimit 2
+- *(1) Filter (isnotnull(latitude#155) AND (latitude#155 = 29.8))
   +- FileScan csv [_c0#150,reportedCity#151,dateOfLoss#152,elevatedBuildingIndicator#153,floodZone#154,latitude#155,longitude#156,lowestFloodElevation#157,amountPaidOnBuildingClaim#158,amountPaidOnContentsClaim#159,yearofLoss#160,reportedZipcode#161,id#162] Batched: false, DataFilters: [isnotnull(latitude#155), (latitude#155 = 29.8)], Format: CSV, Location: InMemoryFileIndex[file:/Users/jeff/Library/Mobile Documents/com~apple~CloudDocs/Documents/jigsaw/..., PartitionFilters: [], PushedFilters: [IsNotNull(latitude), EqualTo(latitude,29.8)], ReadSchema: struct<_c0:int,reportedCity:string,dateOfLoss:timestamp,elevatedBuildingIndicator:boolean,floodZo...




### Resources

[Select and Filter Blog](https://hendra-herviawan.github.io/sparksql-select-filter.html)